In [590]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html
from selenium import webdriver
import time

In [591]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [592]:
url = 'https://torgi.gov.ru/new/api/public/notices/rss?subjRF=50&biddEndFrom=&biddEndTo=&pubFrom=2022-09-13&pubTo=2022-09-13&aucStartFrom=&aucStartTo=&catCode=7&text='
req = requests.get(url, verify=False)

soup = BeautifulSoup(req.text, features='xml')

In [593]:
#soup

In [594]:
num = soup.find_all('item')
description_list = []
for index in num:
    col = index.find(name='description')
    sel = html.unescape(col.text)
    description_list.append(sel)
#print(description_list)

In [595]:
links_all = []
for i in description_list:
    b = BeautifulSoup(i, features='html')
    link = b.find('a')
    link_final = link.get('href')
    links_all.append(link_final)

In [596]:
#links_all

In [597]:
for index in range(0,len(links_all)):
    links_all[index] = 'https://torgi.gov.ru/new/public/lots/lot/' + links_all[index][-20:] + '_1/(lotInfo:info)'
#links_all   

In [598]:
#!pip install webdriver_manager

In [599]:
driver = webdriver.Edge('C:\\Users\\KonovalovAlE\\Desktop\\jupyter notebook\\Вебдрайвер edge\\msedgedriver.exe')

C:\Users\KonovalovAlE\AppData\Local\Temp\ipykernel_13124\171963719.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('C:\\Users\\KonovalovAlE\\Desktop\\jupyter notebook\\Вебдрайвер edge\\msedgedriver.exe')


In [600]:
def data_parse(link):
    driver.get(link)
    time.sleep(2)
    main_page = driver.page_source
    soup = BeautifulSoup(main_page, features='xml')
    
    # Ищу местоположение
    rent = soup.find_all(attrs={"class":{"attr_value"}})
    place = 'Не найдено'
    for i in rent:
        if i.text.find('ул') != -1:
            place = i.text    
        elif 'д.' in i.text and 'Постановление' not in i.text:
            place = i.text
        elif 'на право заключения' in place:
            place = place.split(':')[1]
        elif 'округ' in i.text and 'расположен' not in i.text:
            place = i.text
        elif 'р-н' in i.text and 'по адресу' in i.text:
            place = i.text.split(':')[1]
    # Площадь участка 
    sq = soup.find_all(attrs={"class":{"lotAttributeDetail no-padding-bottom"}})
    square = sq[0].text.split('м')[0].strip()
    if ':' in square:
        square = sq[1].text.split('м')[0].strip()
    
    # Описание лота
    head = soup.find_all(attrs={"class":{"header_title"}})
    header = head[0].text
    
    # Тип имущества
    object_type = ['Нежилые помещения','Здания']
    type_ = soup.find_all(attrs={"class":{"attr_value"}})
    for i in type_:
        if i.text.strip() in object_type:
            type_lot = i.text.strip()
    
    # Начальная цена
    start = soup.find_all(attrs={"class":{"prices__row__price-cell lotPrice"}}) 
    start_price = start[0].text.split('₽')[0]
    
    # Номер извещения
    
    num = soup.find_all(attrs={"class":{"lotAttributeValue"}})
    nums = num[0].text.split(',')
    number = nums[0].split('№')[1]
    
    # Номер лота
    
    lot = nums[1].split('№')
    lot_n = lot[0]+lot[1]
    lot_num = lot_n[1:]
    
    # Ссылка на лот, добавляю в итоговые данные по порядку
    #link которая передаётся в функцию
    
    # Дата окончания приема заявок
    
    date_cl = soup.find_all(attrs={"class":{"lotAttributeValue"}})
    date_close = date_cl[5].text[1:11]
    
    
    # Дата торгов
    
    date_st = soup.find_all(attrs={"class":{"lotAttributeValue"}})
    date_start = date_st[6].text[1:11]
    
    # Количество версий
        #Достаю ссылку на извещение 
    linked = soup.find_all(attrs={"class":{"button text"}})
    link_to = 'https://torgi.gov.ru' + linked[7].get('href')
    link_to
        #Загружаю ссылку через драйвер
    driver.get(link_to)
    time.sleep(2)
    main_page = driver.page_source
    soup = BeautifulSoup(main_page, features='xml')
    
    # Дата публикации 
    lot = soup.find_all(attrs={"class":{"attr_value"}})
    dt_publ = lot[1].text[1:11]
    
    # Организатор торгов
    for i in lot:
        if 'КОМИТЕТ' in i.text:
            auction_org = i.text
        else:
            auction_org = lot[8].text
            if 'КОМИТЕТ' not in auction_org:
                auction_org = lot[10].text
                if auction_org.isdigit():
                    auction_org = lot[8].text
        # Получаю количество версий
    versions = driver.find_element("class name", 'aside-btn')
    versions.click()
    time.sleep(2)
    action_time = driver.find_element("class name", "version-link").get_attribute('innerHTML').strip()
        
    ###############################################
    
    list_of_data = [auction_org,place,square,header,type_lot,start_price,number,lot_num,link,dt_publ,date_close,date_start,action_time]
    
    return list_of_data

In [601]:
#data_parse(links_all[0])

In [602]:
index = ['Организатор торгов','Местоположение','Общая площадь (кв.м)','Описание объекта','Тип имущества','Начальнаяцена,загод','Номер извещения','Номер лота','Ссылка','Дата публикации','Дата окончания приема заявок','Дата торгов/ подведения итогов/отмены','количество версий']
columns = list(range(0,len(links_all)))
data = pd.DataFrame(index=index, columns=columns)

In [603]:
#data

In [604]:
for column in data.columns:
        data[column] = data_parse(links_all[column])


In [605]:
data = data.T

In [606]:
data = data.sort_values(by='количество версий').reset_index(drop=True)

In [607]:
data

,Организатор торгов,Местоположение,Общая площадь (кв.м),Описание объекта,Тип имущества,"Начальнаяцена,загод",Номер извещения,Номер лота,Ссылка,Дата публикации,Дата окончания приема заявок,Дата торгов/ подведения итогов/отмены,количество версий
0,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Московская область, Можайский район, д.Клемент...",9593,Продажа нежилого здания 842 кв.м с земельным у...,Здания,"15407000,00",21000004710000002891,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,13.10.2022,17.10.2022,Версия 1
1,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Московская область, р-н Орехово-Зуевский, г. К...","2251,9","Продажа нежилого здания 357,9 кв.м с земельным...",Здания,"7172531,14",21000004710000002890,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,13.10.2022,17.10.2022,Версия 1
2,КОМИТЕТ ПО КОНКУРЕНТНОЙ ПОЛИТИКЕ МОСКОВСКОЙ ОБ...,"Российская Федерация, Московская область, горо...","1522,1",Продажа объекта незавершенного строительства 1...,Здания,"91838852,00",21000004710000002886,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,14.10.2022,18.10.2022,Версия 1
3,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ВОЙСКОВАЯ ЧАС...","городской округ Чехов, г. Чехов-2 ул. Спортивн...","45,5",Право заключения договора аренды недвижимого и...,Нежилые помещения,"51947,35",21000019850000000003,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,03.10.2022,05.10.2022,Версия 2
4,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ВОЙСКОВАЯ ЧАС...","городской округ Чехов, г. Чехов-2 ул. ул. Севе...","151,5",Право заключения договора аренды недвижимого и...,Нежилые помещения,"362145,60",21000019850000000004,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,03.10.2022,05.10.2022,Версия 2
5,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""РОССИЙСКИЙ АУКЦИОННЫЙ ДОМ""","Московская область, Рузский район, городское п...","35,4","Нежилое помещение: гаражный бокс, площадью 35,...",Нежилые помещения,"409200,00",21000002210000000990,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,29.09.2022,06.10.2022,Версия 2
6,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,"Московская область, Щелковский район, д. Осеев...","1260,9",Аукцион на право заключения договора аренды фе...,Нежилые помещения,"2118312,00",21000010900000000008,лот 1,https://torgi.gov.ru/new/public/lots/lot/21000...,13.09.2022,10.10.2022,12.10.2022,Версия 8


# Сохранение

In [366]:
#data.to_csv('parse_realty_14_09_2022.csv', encoding='utf-8')